In [1]:
import tensorflow as tf
import xlrd

In [2]:
import pandas as pd

In [3]:
d = pd.read_table('dataset.txt')

In [4]:
d[d.isnull().values==True]
d = d.fillna(0)

In [5]:
b = pd.read_table('testset.txt')
b = b.fillna(0)
b.head()

,year,player,age,mp,per,ts,3par,ftr,orb,drb,...,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp,game_win,mvp_label
0,2017,Taurean Waller-Prince,22,59,981,9.8,0.513,0.358,0.330,2.8,...,-0.4,1.5,1.1,0.054,-3.8,1.5,-2.3,-0.1,50,0.0
1,2017,Thabo Sefolosha,32,62,1596,11.9,0.519,0.304,0.190,3.8,...,0.6,2.7,3.3,0.098,-1.9,3.1,1.2,1.3,50,0.0
2,2017,Mike Scott,28,18,195,5.9,0.360,0.458,0.136,5.7,...,-0.3,0.2,-0.1,-0.014,-5.3,-0.1,-5.3,-0.2,50,0.0
3,2017,Dennis Schroder,23,79,2485,16.1,0.533,0.242,0.210,1.9,...,1.2,2.5,3.7,0.071,0.7,-1.4,-0.7,0.8,50,0.0
4,2017,Lamar Patterson,25,5,40,-1.9,0.276,0.400,0.200,2.8,...,-0.2,0.0,-0.1,-0.176,-9.9,-1.2,-11.2,-0.1,50,0.0


In [6]:
import numpy as np

In [7]:
temp = d.values[:,0]
temp1 = b.values[:,0]

In [8]:
temp = np.reshape(temp, (-1,1))
temp1 = np.reshape(temp1, (-1,1))
a = np.hstack((temp, d.values[:,2:]))
t = np.hstack((temp1, b.values[:,2:]))
# t = np.hstack((temp1, b.values[:, 2:16]))

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

In [10]:
transform = Normalizer().fit(a[:,1:25])
new_a = transform.transform(a[:,1:25])
transform = Normalizer().fit(t[:,1:25])
new_t = transform.transform(t[:,1:25])

In [11]:
a_ = np.hstack((a[:,0].reshape(-1,1), new_a))
a = np.hstack((a_, a[:,25].reshape(-1,1)))
a = np.hstack((a, d.values[:,-1].reshape(-1,1)))

In [12]:
t_ = np.hstack((t[:,0].reshape(-1,1), new_t))
t = np.hstack((t_, t[:,25].reshape(-1,1)))
t = np.hstack((t, b.values[:,-1].reshape(-1,1)))

In [13]:
x = tf.placeholder(dtype=tf.float32, shape=([None, 25]), name='input')
y_true = tf.placeholder(dtype=tf.float32, shape=([None,1]), name='mvp')

class Helper():
    
    def __init__(self, data, test_data):
        self.i = 0
        self.all_train_batches = data
        self.test_images = test_data[:,:25]
        self.test_labels = test_data[:,25]
    
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

# def conv2d(x, W):
#     return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# def max_pool_2by2(x):
#     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
#                           strides=[1, 2, 2, 1], padding='SAME')

# def convolutional_layer(input_x, shape):
#     W = init_weights(shape)
#     b = init_bias([shape[3]])
#     return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b



In [14]:
s = [0]*31
sum_ = 0
print(len(temp))
for year in temp:
    s[int(year)-1986] += 1
for i in range(len(s)):
    s[i] += sum_
    sum_ = s[i]
    
tt = [0]*3
sum_ = 0
print(len(temp1))
for year in temp1:
    tt[int(year)-2017] += 1
for i in range(len(tt)):
    tt[i] += sum_
    sum_ = tt[i]


out1 = normal_full_layer(x,500)
layer1 = tf.tanh(out1)

out2 = normal_full_layer(layer1, 100)
layer2 = tf.tanh(out2)

out3 = normal_full_layer(layer2, 1)
y_pred = tf.sigmoid(out3)

mse = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_true,predictions=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00012)
train = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5000):
        loopNum = i%20
        if loopNum == 0:
            _, loss, y_hat = sess.run([train,mse,y_pred], feed_dict={x: a[:s[loopNum], 1:26], y_true: a[:s[loopNum], 26].reshape(-1,1)})
        else:
            _, loss, y_hat = sess.run([train,mse, y_pred], feed_dict={x: a[s[loopNum-1]:s[loopNum], 1:26], y_true: a[s[loopNum-1]:s[loopNum], 26].reshape(-1,1)})
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('loss is:{:f}'.format(loss))
            # print(y_hat)
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            accuracy, y_hat2 = sess.run([acc,y_pred],feed_dict={x:t[:,1:26],y_true:t[:,26].reshape(-1,1)})
            print('\n')
    print(accuracy)

15059
1547
Currently on step 0
loss is:0.174851


Currently on step 100
loss is:0.005453


Currently on step 200
loss is:0.003448


Currently on step 300
loss is:0.002716


Currently on step 400
loss is:0.001938


Currently on step 500
loss is:0.000996


Currently on step 600
loss is:0.000518


Currently on step 700
loss is:0.000325


Currently on step 800
loss is:0.000227


Currently on step 900
loss is:0.000169


Currently on step 1000
loss is:0.000131


Currently on step 1100
loss is:0.000105


Currently on step 1200
loss is:0.000086


Currently on step 1300
loss is:0.000071


Currently on step 1400
loss is:0.000060


Currently on step 1500
loss is:0.000052


Currently on step 1600
loss is:0.000045


Currently on step 1700
loss is:0.000039


Currently on step 1800
loss is:0.000034


Currently on step 1900
loss is:0.000030


Currently on step 2000
loss is:0.000027


Currently on step 2100
loss is:0.000024


Currently on step 2200
loss is:0.000022


Currently on step 2300
loss is:0.00

In [15]:
b.values[np.argpartition(y_hat2[:,0], -5)[-5:],0:2]

array([[2018, 'Naz Mitrou-Long'],
       [2019, 'James Harden'],
       [2018, 'James Harden'],
       [2019, 'Giannis Antetokounmpo'],
       [2017, 'Russell Westbrook']], dtype=object)

In [16]:
y_hat2[np.argpartition(y_hat2[:,0], -5)[-5:]]

array([[ 0.00402026],
       [ 0.18727021],
       [ 0.9859497 ],
       [ 0.18131027],
       [ 0.98605448]], dtype=float32)

In [17]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())

#     for i in range(5000):
#         loopNum = i%20
#         sess.run(train, feed_dict={x: batch[0], y_true: batch[1]})
        
#         # PRINT OUT A MESSAGE EVERY 100 STEPS
#         if i%100 == 0:
            
#             print('Currently on step {}'.format(i))
#             print('Accuracy is:')
#             # Test the Train Model
#             matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))

#             acc = tf.reduce_mean(tf.cast(matches,tf.float32))

#             print(sess.run(acc,feed_dict={x:,y_true:ch.test_labels,hold_prob:1.0}))
#             print('\n')


In [18]:
# temp1